# creation of the table *tower_hot_201308*

```{sql}
CREATE TABLE sight_xy_201308 (
	SELECT A.SERV_ID, A.ACC_NBR, A.MONTH_NO,
		SUBSTRING_INDEX(A.START_TIME,' ',1) AS DATE, DAYOFWEEK(SUBSTRING_INDEX(A.START_TIME,' ',1)) AS WEEK_DAY,
		CONVERT(SUBSTRING_INDEX(SUBSTRING_INDEX(A.START_TIME,' ',-1),':',1),UNSIGNED) AS HOUR_NO,
		A.LAC, A.CELL_ID, B.LON, B.LAT, B.X, B.Y
	FROM serv_acct_item_0838_201308 A JOIN tower B
	ON A.CELL_ID=B.CELL_16
	WHERE 
        A.LAC!='-1' AND 
        A.LAC!='0' AND 
        A.START_TIME LIKE '____/%/% %:__:__'
);
CREATE INDEX xy_sight_201308 ON sight_xy_201308(LON, LAT);
CREATE INDEX weekhour_xy_201308 ON sight_xy_201308(WEEK_DAY, HOUR_NO);
```

Basically, there are three types of data in this database which are user
information, tower information and calling records. The table *sight_xy_201308*
is created by jointing calling records with the tower informatin on *cell*.
We can also observe that the calling records' *CELL_ID* share similarity with
tower information's *CELL_16*. Moreover, outliers include -1, 0 in *LAC*
and the format of *START_TIME* should be in a specific format. More information
of *LAC(location area code)* can be found [here]().

```{sql}
CREATE TABLE sight_weekend_201308 (
	SELECT LON, LAT, COUNT(1) AS REC_NO
	FROM sight_xy_201308
	WHERE (WEEK_DAY=1 OR WEEK_DAY=7) AND (HOUR_NO>=10 AND HOUR_NO<22)
	GROUP BY LON, LAT
	ORDER BY COUNT(1) DESC
);
ALTER TABLE sight_weekend_201308 ADD RANK INT AUTO_INCREMENT KEY;
ALTER TABLE sight_weekend_201308 ADD CENTER_FLAG INT; 
UPDATE sight_weekend_201308 SET CENTER_FLAG=0;
SET @n=(
	SELECT COUNT(1) FROM sight_weekend_201308
);
UPDATE sight_weekend_201308 SET CENTER_FLAG=1
	WHERE RANK<=0.01*@n;
CREATE INDEX xy_weekend_201308 ON sight_weekend_201308(LON, LAT);
```

*note: The command `Count(1)` is euqivalent to `Count(*)` returning the number of 
observatons in a column.*

```{sql}
CREATE TABLE tower_hot_201308 (
	SELECT 
        -- coluns from tower table
        A.CELL_10, A.CELL_16, A.LON, A.LAT, A.X, A.Y,
        -- create MAIN_CENTER_FLAG column
        -- 
        A.DEYANG_CENTER AS MAIN_CENTER_FLAG, 
		IFNULL(B.CENTER_FLAG,0) AS HOT_FLAG, 
        CONVERT(A.DEYANG_CENTER,UNSIGNED)*IFNULL(B.CENTER_FLAG,0) AS HOT_MAIN_FLAG, 
		IFNULL(B.CENTER_FLAG,0)-CONVERT(A.DEYANG_CENTER,UNSIGNED)*IFNULL(B.CENTER_FLAG,0) AS HOT_OTHER_FLAG
	FROM tower A LEFT JOIN sight_weekend_201308 B
	ON A.LON=B.LON AND A.LAT=B.LAT
);
CREATE INDEX cell_tower_201308 ON tower_hot_201308(CELL_16);
```

The structure of the *tower_hot_201308* is inherited from the *tower* and 
*sight_weekend_201308*. Two tables are jointed on the longitude and latitude.
Apart from the inherited columns, there are some new columns such as HOT_FLAG, 
HOT_MAIN_FLAG, HOT_OTHER_FLAG. To inspect the meaning, we have to first understand
how *sight_weekend_201308* is created